Our IP formulation works as such:

$s_{i}$: the start time of task $i$

$d_{i}$: the delay past finish time of task $i$

$y_{ij}$: 1 if task $i$ before task $j$, 0 if not

$z$: 1 if a hot-cold-hot sequence occurs, 0 if not

Our formulated model for this problem is:

minimize $\hspace{10pt} rz + \sum_{i \in T} d_i$

such that $\hspace{10pt} d_i \geq s_i + p_i - f_i \hspace{54pt} \forall i \in T$

$\hspace{42pt} s_i - s_j \leq -p_i + M(1-y_{ij}) \hspace{15pt} \forall i \in T, j \in T; i<j$

$\hspace{42pt} s_j - s_i \leq -p_j + My_{ij} \hspace{35pt} \forall i \in T, j \in T; i<j$

$\hspace{42pt} z \geq y_{12} + y_{23} - 1$

$\hspace{42pt} z \geq (1 - y_{12}) + (1 - y_{23}) - 1$

$\hspace{42pt} s_i, d_i \geq 0 \hspace{81pt} \forall i \in T$

$\hspace{42pt} y_{i,j} \in \{0, 1\} \hspace{71pt} \forall i \in T, j \in T; i<j$

$\hspace{42pt} z \in \{0, 1\}$

In [9]:
using Cbc, JuMP, NamedArrays

# Sets
T = 1:6
H = [1,3]
C = [2]

# Parameters
p = Dict(zip(T, [10, 7, 12, 8, 5, 14]))
f = Dict(zip(T, [32, 22, 15, 22, 13, 36]))
r = 20
M = 56

########################################
# TODO: create and solve general model #
########################################

m = Model()

@variable(m, s[T] >= 0)                           # start time of task t
@variable(m, d[T] >= 0)                           # delay past finish time of task t
@variable(m, y[i in T, j in T; i < j], Bin)       # binary variable; 1 if task i before j, 0 if not
@variable(m, z, Bin)                              # binary variable; 1 if hot-cold-hot occurs, 0 if not

@objective(m, Min, sum(d[i] for i in T) + r*z)

@constraint(m, delay[i in T], d[i] >= s[i] + p[i] - f[i])
@constraint(m, ordering1[i in T, j in T; i < j], s[i] - s[j] <= -p[i] + M*(1-y[i,j]))
@constraint(m, ordering2[i in T, j in T; i < j], s[j] - s[i] <= -p[j] + M*y[i,j])'
@constraint(m, hotcoldhot1, z >= y[1,2] + y[2,3] - 1)
@constraint(m, hotcoldhot2, z >= (1 - y[1,2]) + (1 - y[2,3]) - 1)

set_optimizer(m, Cbc.Optimizer)

optimize!(m)

solution_summary(m, verbose=true)

Welcome to the CBC MILP Solver 
Version: 2.10.5 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 0 - 0.00 seconds
Cgl0004I processed model has 38 rows, 28 columns (16 integer (16 of which binary)) and 108 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 15 integers unsatisfied sum - 2.67857
Cbc0038I Pass   1: suminf.    0.00000 (0) obj. 86 iterations 24
Cbc0038I Solution found of 86
Cbc0038I Relaxing continuous gives 86
Cbc0038I Before mini branch and bound, 1 integers at bound fixed and 2 continuous
Cbc0038I Full problem 38 rows 28 columns, reduced to 27 rows 20 columns
Cbc0038I Mini branch and bound improved solution from 86 to 75 (0.05 seconds)
Cbc0038I Round again with cutoff of 66.6001
Cbc0038I Pass   2: suminf.    0.06929 (1) obj. 66.6001 iterations 2
Cbc0038I Pass   3: suminf.    0.05982 (1) obj. 66.6001 iterations 5
Cbc0038I Pass   4: suminf.    0.05000 (1) obj. 66.6001 ite

* Solver : COIN Branch-and-Cut (Cbc)

* Status
  Termination status : OPTIMAL
  Primal status      : FEASIBLE_POINT
  Dual status        : NO_SOLUTION
  Result count       : 1
  Has duals          : false
  Message from the solver:
  "Cbc_status          = finished - check isProvenOptimal or isProvenInfeasible to see if solution found (or check value of best solution)
Cbc_secondaryStatus = search completed with solution
"

* Candidate solution
  Objective value      : 47.0
  Objective bound      : 47.0
  Primal solution :
    d[1] : 10.0
    d[2] : 0.0
    d[3] : 16.999999999999996
    d[4] : 0.0
    d[5] : 0.0
    d[6] : 20.0
    s[1] : 32.0
    s[2] : 5.000000000000001
    s[3] : 19.999999999999996
    s[4] : 11.999999999999996
    s[5] : 0.0
    s[6] : 42.0
    y[1,2] : 0.0
    y[1,3] : 0.0
    y[1,4] : 0.0
    y[1,5] : 0.0
    y[1,6] : 1.0
    y[2,3] : 1.0
    y[2,4] : 1.0
    y[2,5] : 0.0
    y[2,6] : 1.0
    y[3,4] : 0.0
    y[3,5] : 0.0
    y[3,6] : 1.0
    y[4,5] : 0.0
    y[4,